In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import numpy as np
import pandas as pd
import gzip
import json

from pprint import pprint
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

#Preprocessing

In [3]:
nltk.download('stopwords')
turkish_stopwords = stopwords.words('turkish')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [4]:
my_classification_df = pd.read_csv("/content/drive/MyDrive/CS412 project/my_annotations.csv",)
my_classification_df = my_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'influencerCategory': 'category'})
train_classification_df = pd.read_csv("/content/drive/MyDrive/CS412 project/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'Unnamed: 0': 'user_id', 'label': 'category'})
train_classification_df.loc[train_classification_df['category'] == 'Health and lifestyle', 'category'] = 'Health and Lifestyle'

# Combine datasets
combined_df = pd.concat([train_classification_df, my_classification_df], ignore_index=True)

# Drop rows where 'category' is NaN or user_id is missing
combined_df.dropna(subset=['user_id', 'category'], inplace=True)

# Ensure all categories are strings
combined_df['category'] = combined_df['category'].astype(str).str.strip()

# Create unified dictionary
username2_category = combined_df.set_index("user_id")["category"].to_dict()

In [5]:
combined_df.groupby("category").count()

,user_id,url,influencerMention,accountType
category,,,,
Art,214,23,23,23
Entertainment,340,17,17,17
Fashion,309,10,10,10
Food,526,15,15,15
Gaming,19,6,6,6
Health and Lifestyle,525,22,22,22
Mom and Children,161,12,12,12
Sports,130,17,17,17
Tech,359,13,13,13


In [6]:
train_data_path = "/content/drive/MyDrive/CS412 project/training-dataset.jsonl.gz"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with gzip.open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]
    if username in username2_category:
      # train data info
      username2posts_train[username] = sample["posts"]
      username2profile_train[username] = profile


    else:
      # it is test data info
      username2posts_test[username] = sample["posts"]
      username2profile_test[username] = profile


In [7]:
# Profile Dataframe
train_profile_df = pd.DataFrame(username2profile_train).T.reset_index(drop=True)
test_profile_df = pd.DataFrame(username2profile_test).T.reset_index(drop=True)

train_profile_df.head(2)

,username,id,full_name,biography,category_name,post_count,follower_count,following_count,is_business_account,is_private,...,business_category_name,overall_category_name,category_enum,is_verified_by_mv4b,is_regulated_c18,profile_pic_url,should_show_category,should_show_public_contacts,show_account_transparency_details,profile_picture_base64
0,deparmedya,3170700063,Depar Medya,#mediaplanning #mediabuying #sosyalmedya,Local business,None,1167,192,True,False,...,None,None,LOCAL,False,False,https://instagram.fsaw2-3.fna.fbcdn.net/v/t51....,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...
1,kafesfirin,266439571,KAFES FIRIN,📍Söğütözü📍FTZ AVM\n🛒Ankara macro▲center v...,Brand,None,11997,17,True,False,...,None,None,BRAND,False,False,https://instagram.fada1-13.fna.fbcdn.net/v/t51...,True,True,True,/9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...


In [8]:
# Preprocess captions (keep emojis intact)
def preprocess_text(text: str):
    text = text.casefold()
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)
    text = re.sub(r'[^a-zçğıöşü0-9\s#@]', '', text)
    text = re.sub(r'\d+', '', text)        # Remove numbers
    text = re.sub(r'\s+', ' ', text).strip()
    return text


corpus = []

# to keep the label order
train_usernames = []

# Aggregate posts per user
for username, posts in username2posts_train.items():
    train_usernames.append(username)
    cleaned_captions = []
    for post in posts:
        post_caption = post.get("caption", "")
        if post_caption is None:
            continue

        post_caption = preprocess_text(post_caption)
        if post_caption != "":
            cleaned_captions.append(post_caption)

    user_post_captions = " \n ".join(cleaned_captions)
    corpus.append(user_post_captions)




#BERT model
Discared due to long training time despite having around the same accuracy as random forest

In [9]:
# Tokenize data using BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("dbmdz/bert-base-turkish-uncased")
train_encodings = tokenizer(corpus, truncation=True, padding=True, max_length=128, return_tensors="tf")

# Prepare labels
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/263k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

In [10]:
# Extract components as numpy arrays
input_ids = train_encodings["input_ids"].numpy()
attention_mask = train_encodings["attention_mask"].numpy()
token_type_ids = train_encodings["token_type_ids"].numpy()

# Split each component and the labels
x_train_input_ids, x_val_input_ids, x_train_attention_mask, x_val_attention_mask, x_train_token_type_ids, x_val_token_type_ids, y_train_encoded, y_val_encoded = train_test_split(
    input_ids, attention_mask, token_type_ids, y_train_encoded, test_size=0.2, stratify=y_train_encoded, random_state=42
)

# Combine the split components into dictionaries
x_train_encodings = {
    "input_ids": x_train_input_ids,
    "attention_mask": x_train_attention_mask,
    "token_type_ids": x_train_token_type_ids,
}

x_val_encodings = {
    "input_ids": x_val_input_ids,
    "attention_mask": x_val_attention_mask,
    "token_type_ids": x_val_token_type_ids,
}


In [11]:
# Load BERT model for sequence classification
model = TFBertForSequenceClassification.from_pretrained("dbmdz/bert-base-turkish-uncased", num_labels=len(label_encoder.classes_))

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = [tf.keras.metrics.SparseCategoricalAccuracy("accuracy")]

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

tf_model.h5:   0%|          | 0.00/545M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at dbmdz/bert-base-turkish-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
history = model.fit(
    x={
        "input_ids": x_train_encodings["input_ids"],
        "attention_mask": x_train_encodings["attention_mask"],
        "token_type_ids": x_train_encodings["token_type_ids"],
    },
    y=y_train_encoded,
    validation_data=(
        {
            "input_ids": x_val_encodings["input_ids"],
            "attention_mask": x_val_encodings["attention_mask"],
            "token_type_ids": x_val_encodings["token_type_ids"],
        },
        y_val_encoded,
    ),
    epochs=3,
    batch_size=16,
)


Epoch 1/3
  9/139 [>.............................] - ETA: 1:03:24 - loss: 2.2011 - accuracy: 0.2153

KeyboardInterrupt: 

In [ ]:
# Evaluate the model
val_loss, val_accuracy = model.evaluate(x_val_encodings, y_val_encoded)
print(f"Validation Accuracy: {val_accuracy:.4f}")

# Classification Report
y_val_pred = np.argmax(model.predict(x_val_encodings).logits, axis=1)
print("\nClassification Report:\n")
print(classification_report(y_val_encoded, y_val_pred, target_names=label_encoder.classes_))


#Random Forest

In [13]:
vectorizer = TfidfVectorizer(stop_words=turkish_stopwords, max_features=5000)
vectorizer.fit(corpus)

TfidfVectorizer(max_features=5000,
                stop_words=['acaba', 'ama', 'aslında', 'az', 'bazı', 'belki',
                            'biri', 'birkaç', 'birşey', 'biz', 'bu', 'çok',
                            'çünkü', 'da', 'daha', 'de', 'defa', 'diye', 'eğer',
                            'en', 'gibi', 'hem', 'hep', 'hepsi', 'her', 'hiç',
                            'için', 'ile', 'ise', 'kez', ...])

In [14]:
# transform the data into vectors
x_post_train = vectorizer.transform(corpus)
y_train = [username2_category.get(uname, "NA") for uname in train_usernames]


test_usernames = []
test_corpus = []
for username, posts in username2posts_test.items():
  test_usernames.append(username)
  # aggregating the posts per user
  cleaned_captions = []
  for post in posts:
    post_caption = post.get("caption", "")
    if post_caption is None:
      continue

    post_caption = preprocess_text(post_caption)

    if post_caption != "":
      cleaned_captions.append(post_caption)

  user_post_captions = "\n".join(cleaned_captions)
  test_corpus.append(user_post_captions)


# Just transforming! No Fitting!!!!!
x_post_test = vectorizer.transform(test_corpus)

In [15]:
assert y_train.count("NA") == 0

In [16]:
feature_names = vectorizer.get_feature_names_out()
feature_names

array(['abdullah', 'abone', 'about', ..., 'şık', 'şıklık', 'şıklığı'],
      dtype=object)

In [17]:
df_tfidf = pd.DataFrame(x_post_train.toarray(), columns=feature_names)
df_tfidf.head(2)

,abdullah,abone,about,acele,acil,activities,acı,ad,ada,adam,...,şubemiz,şubesi,şölen,şöleni,şöyle,şükranla,şükür,şık,şıklık,şıklığı
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.048769,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [18]:
df_tfidf.shape

(2780, 5000)

In [19]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(df_tfidf, y_train, test_size=0.2, stratify=y_train ,random_state=42)

In [20]:
x_train.shape

(2224, 5000)

In [21]:
x_val.shape

(556, 5000)

#User Classification
We use Random Forest along with vectorized contents to predict influencer type.

In [22]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(RandomForestClassifier(), param_grid, scoring='accuracy', cv=5, n_jobs=-1)
grid_search.fit(x_train, y_train)

best_model = grid_search.best_estimator_
print("Best Parameters:", grid_search.best_params_)


Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 300}


In [23]:
rf_model = RandomForestClassifier(
    n_estimators=200,  # Number of trees
    random_state=42,   # For reproducibility
    max_depth=None,    # Expand trees until all leaves are pure or contain < min_samples_split samples
    min_samples_split=10,  # Minimum samples required to split an internal node
    n_jobs=-1          # Use all processors for parallel training
)

# Train the model
rf_model.fit(x_train, y_train)

RandomForestClassifier(min_samples_split=10, n_estimators=200, n_jobs=-1,
                       random_state=42)

In [24]:
# Evaluate on training set
y_train_pred= rf_model.predict(x_train)
print("Training Accuracy:", accuracy_score(y_train, y_train_pred))
print("\nTraining Classification Report:")
print(classification_report(y_train, y_train_pred))



Training Accuracy: 0.9973021582733813

Training Classification Report:
                      precision    recall  f1-score   support

                 Art       1.00      0.99      1.00       157
       Entertainment       1.00      1.00      1.00       262
             Fashion       1.00      1.00      1.00       241
                Food       0.99      1.00      0.99       412
              Gaming       1.00      1.00      1.00        12
Health and Lifestyle       1.00      1.00      1.00       408
    Mom and Children       1.00      1.00      1.00       118
              Sports       1.00      1.00      1.00        97
                Tech       1.00      1.00      1.00       280
              Travel       1.00      1.00      1.00       237

            accuracy                           1.00      2224
           macro avg       1.00      1.00      1.00      2224
        weighted avg       1.00      1.00      1.00      2224



In [25]:
# Evaluate on validation set
y_val_pred = rf_model.predict(x_val)
print("Validation Accuracy:", accuracy_score(y_val, y_val_pred))
print("\nValidation Classification Report:")
print(classification_report(y_val, y_val_pred))

Validation Accuracy: 0.5935251798561151

Validation Classification Report:
                      precision    recall  f1-score   support

                 Art       1.00      0.05      0.10        40
       Entertainment       0.30      0.28      0.29        65
             Fashion       0.63      0.73      0.68        60
                Food       0.75      0.92      0.83       103
              Gaming       0.00      0.00      0.00         3
Health and Lifestyle       0.45      0.72      0.56       102
    Mom and Children       0.75      0.10      0.18        29
              Sports       1.00      0.17      0.29        24
                Tech       0.66      0.81      0.73        70
              Travel       0.83      0.57      0.67        60

            accuracy                           0.59       556
           macro avg       0.64      0.43      0.43       556
        weighted avg       0.65      0.59      0.55       556



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


##Test Data

In [49]:
def test_classification(input_file,output_file):
  #open_file
  test_data_path = "/content/drive/MyDrive/CS412 project/"+input_file
  test_unames = []
  with open(test_data_path, "rt") as fh:
    for line in fh:
      test_unames.append(line.strip())
  x_test = []
  if "screenname" in test_unames:
    test_unames.remove("screenname")
  for uname in test_unames:
    try:
      index = test_usernames.index(uname)
      x_test.append(x_post_test[index].toarray()[0])
    except Exception as e:
      try:
        index = train_usernames.index(uname)
        x_test.append(x_post_train[index].toarray()[0])
      except Exception as e:
        print(uname)
  df_test = pd.DataFrame(np.array(x_test), columns=feature_names)
  #make predictions
  test_pred = rf_model.predict(df_test)
  output = dict()
  for index, uname in enumerate(test_unames):
    output[uname] = test_pred[index]
  with open(output_file, "w") as of:
    json.dump(output, of, indent=4)

In [50]:
test_classification("test-classification-round1.dat","prediction-classification-round1.json")
test_classification("test-classification-round2.dat","prediction-classification-round2.json")
test_classification("test-classification-round3.dat","prediction-classification-round3.json")

# Like Count Prediction


Here, we use train a linear regression model with log like counts - avg log like counts(per user). I didn't use vectorized content for this as that would only confuse the model. Since we're trying to minimize log MSE, this seemed to be the best approach.

In [31]:
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import Ridge
from scipy.sparse import hstack
from datetime import datetime


In [32]:
# Get average like count for a user
def get_log_avg_like_count(posts):
    like_counts = [post.get("like_count", 0)  if post.get("like_count") is not None else 0 for post in posts]
    log_like_counts = np.log1p(like_counts)
    if not like_counts:
        return 0
    return sum(log_like_counts) / len(like_counts)

In [33]:
# Extract additional features from posts
def extract_features(post):
    # Hashtag count
    caption = post.get("caption", "") or ""
    hashtag_count = caption.count("#")

    # Caption length (in words)
    caption_length = len(caption.split())

    # Comments count
    comments_count = post.get("comments_count", 0)

    # Media type (one-hot encoded)
    media_type = post.get("media_type", "IMAGE")
    media_type_encoded = [1 if media_type == "IMAGE" else 0, 1 if media_type == "VIDEO" else 0]

    # Timestamp features
    timestamp = post.get("timestamp", "")
    if timestamp:
        dt = datetime.strptime(timestamp, "%Y-%m-%d %H:%M:%S")
        hour = dt.hour
        day_of_week = dt.weekday()
        month = dt.month
    else:
        hour = day_of_week = month = 0

    return [hashtag_count, caption_length, comments_count, hour, day_of_week, month] + media_type_encoded


In [34]:
# Prepare training data
features = []
log_avg_like_counts = []
log_like_counts = []
like_counts = []
for username, posts in username2posts_train.items():
    log_avg_like_count = get_log_avg_like_count(posts)
    for post in posts:
        like_count = post.get("like_count")
        features.append(extract_features(post))
        log_avg_like_counts.append(log_avg_like_count)
        if like_count is None:
          like_count = 0
        log_like_counts.append(np.log1p(like_count))
        like_counts.append(like_count)

In [35]:
# Convert features to numpy array
x_train = np.array(features)
log_avg_like_counts = np.array(log_avg_like_counts)
log_like_counts = np.array(log_like_counts)


In [36]:
from sklearn.linear_model import LinearRegression

# Train a linear regression model
regressor = LinearRegression()
regressor.fit(x_train, log_like_counts - log_avg_like_counts)

# Evaluate the model
y_train_pred_log_diff = regressor.predict(x_train)
y_train_pred_log = y_train_pred_log_diff + log_avg_like_counts
log_mse = mean_squared_error(log_like_counts, y_train_pred_log)
print(f"Training Log MSE: {log_mse}")

Training Log MSE: 0.7143595185506122


In [37]:
# Predict like count for a post
def predict_like_count(username, current_post=None):
    if username not in username2posts_train and username not in username2posts_test:
        print(f"No data available for {username}")
        return 0

    # Get the user's log average like count
    if username in username2posts_train:
        log_avg_like_count = get_log_avg_like_count(username2posts_train[username])
    elif username in username2posts_test:
        log_avg_like_count = get_log_avg_like_count(username2posts_test[username])

    # Extract features for the current post
    current_features = np.array([extract_features(current_post)])

    # Predict log-like count difference
    predicted_log_diff = regressor.predict(current_features)[0]

    # Reverse log transformation
    predicted_log_like_count = predicted_log_diff + log_avg_like_count
    if predicted_log_like_count < 0:
      predicted_log_like_count = 0
    predicted_like_count = np.expm1(predicted_log_like_count)

    return predicted_like_count

In [38]:
def log_mse_like_counts(y_true, y_pred):
  """
  Calculate the Log Mean Squared Error (Log MSE) for like counts (log(like_count + 1)).

  Parameters:
  - y_true: array-like, actual like counts
  - y_pred: array-like, predicted like counts

  Returns:
  - log_mse: float, Log Mean Squared Error
  """
  # Ensure inputs are numpy arrays
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)

  # Log transformation: log(like_count + 1)
  log_y_true = np.log1p(y_true)
  log_y_pred = np.log1p(y_pred)

  # Compute squared errors
  squared_errors = (log_y_true - log_y_pred) ** 2

  # Return the mean of squared errors
  return np.mean(squared_errors)

In [39]:
#@title Train Dataset evaluation

y_like_count_train_true = []
y_like_count_train_pred = []
for uname, posts in username2posts_train.items():
  for post in posts:
    pred_val = predict_like_count(uname, post)
    true_val = post.get("like_count", 0)
    if true_val is None:
      true_val = 0

    y_like_count_train_true.append(true_val)
    y_like_count_train_pred.append(pred_val)

print(f"Log MSE Train= {log_mse_like_counts(y_like_count_train_true, y_like_count_train_pred)}")

Log MSE Train= 0.7143189830483252


In [52]:
#@title Test Dataset
def test_regression(input_file,output_file):
  path = "/content/drive/MyDrive/CS412 project/"+input_file
  output_path = "/content/"+output_file

  to_predict_like_counts_usernames = []
  output_list = []
  with open(path, "rt") as fh:
    for line in fh:
      sample = json.loads(line)

      # let's predict
      pred_val = predict_like_count(sample["username"],sample)
      sample["like_count"] = int(pred_val)
      output_list.append(sample)

  with open(output_path, "wt") as of:
    json.dump(output_list, of)

In [55]:
test_regression("test-regression-round1.jsonl","prediction-regression-round1.json")
test_regression("test-regression-round2.jsonl","prediction-regression-round2.json")
test_regression("test-regression-round3.jsonl","prediction-regression-round3.json")